# 🏠 Sarajevo Flats Scraper
This notebook demonstrates how to collect real estate data (flats in Sarajevo Canton) from **NEKRETNINE.ba**, a popular Bosnian classifieds platform.

The goal is to:
- Collect key property details (title, price, size, location, condition…)
- Store them in a structured dataset (`sarajevo_flats.csv`)
- Prepare the dataset for future analysis or machine learning (e.g. AI price estimation)

We'll use **Selenium** for dynamic page loading and **BeautifulSoup** for parsing HTML.


In [ ]:
import sys
import os

print("=" * 80)
print("PYTHON ENVIRONMENT INFO")
print("=" * 80)
print(f"\nPython executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"\nPython path (where packages are loaded from):")
for i, path in enumerate(sys.path[:5], 1):
    print(f"  {i}. {path}")
print("\n" + "=" * 80)

: 

In [ ]:
import os
import time
import csv
import re
import random
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import WebDriverException, TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

In [ ]:
# Firefox + Geckodriver setup
firefox_binary = "/usr/bin/firefox"
geckodriver_binary = "/home/mustafasinanovic/miniforge3/bin/geckodriver"

# Scraper settings
BASE_URL = "https://nekretnine.ba/listing.php?lang=ba&sel=nekretnine&grad=65&naselje=&kat=3&subjekt=2&cij1=&cij2=&pov1=&pov2=&spr1=&spr2=&firma=&page={}"
OUTPUT_CSV = "data/sarajevo_flats_nekretnine.csv"
MAX_PAGES = 88
REQUEST_DELAY = (2, 5)

# Multithreading settings
MAX_WORKERS = 3  # Number of parallel browser instances (don't set too high to avoid blocking)

os.makedirs("data", exist_ok=True)

The scraper will fetch up to 88 pages of listings from the OLX search results for *Sarajevo Canton flats*.  
All results are stored in `data/sarajevo_flats_nekretnine.csv`.  
We use randomized delays between requests to reduce the risk of blocking.

In [ ]:
def clean_text(s):
    return " ".join(s.split()).strip() if s else None

def extract_price(text):
    if not text:
        return None
    cleaned = re.sub(r"[^0-9]", "", text)
    return int(cleaned) if cleaned else None

def extract_number(text):
    if not text:
        return None
    m = re.search(r"(\d+)", text)
    return int(m.group(1)) if m else None

In [ ]:
def fetch_page_source(url, driver, short_wait=10):
    """
    Loads a given URL and returns whatever HTML is available immediately.
    Does NOT wait for the page to fully load (useful for slow or problematic websites).

    Parameters:
        url (str): URL to load
        driver (webdriver): Selenium WebDriver instance
        short_wait (int or float): seconds to wait after opening page before returning source

    Returns:
        str or None: HTML source (may be partially loaded)
    """
    try:
        print(f"[+] Attempting to load URL quickly: {url}")
        driver.get(url)
        time.sleep(short_wait)  # minimal wait to let some content render
        html = driver.page_source
        if html:
            print(f"[+] HTML fetched (may be partial): {url}")
        else:
            print(f"[!] No HTML returned for {url}")
        return html
    except (TimeoutException, WebDriverException, OSError) as e:
        print(f"[!] Failed to load page: {url} → {e}")
        return None
    except Exception as e:
        print(f"[!] Unexpected error loading page: {url} → {e}")
        return None


This function uses Selenium to load pages dynamically.
If a page fails (timeout, network error, etc.), we log the issue but continue scraping.

In [ ]:
def parse_detail_page(url, driver):
    html = fetch_page_source(url, driver)
    if not html:
        return None

    try:
        soup = BeautifulSoup(html, "lxml")

        # Extract title
        title_elem = soup.select_one("div.listing-titlebar-title h2")
        if title_elem:
            # Remove the tag span from title
            tag_span = title_elem.find("span", class_="listing-tag")
            if tag_span:
                tag_span.decompose()
            title = clean_text(title_elem.get_text())
        else:
            title = None

        # Extract municipality (address/location)
        municipality_elem = soup.select_one("a.listing-address")
        municipality = clean_text(municipality_elem.get_text()) if municipality_elem else None

        # Extract price
        price_elem = soup.select_one("span.re-slidep")
        price_numeric = extract_price(price_elem.get_text()) if price_elem else None

        # Extract property type
        property_type_elem = soup.find("b", string="TIP")
        property_type = clean_text(property_type_elem.find_next("div").get_text()) if property_type_elem else None

        # Extract ad type (subject - prodaja/izdavanje)
        ad_type_elem = soup.find("b", string="SUBJEKT")
        ad_type = clean_text(ad_type_elem.find_next("div").get_text()) if ad_type_elem else None

        # Extract rooms
        rooms_elem = soup.find("b", string="BROJ SOBA")
        rooms = clean_text(rooms_elem.find_next("div").get_text()) if rooms_elem else None

        # Extract square meters
        square_m2_elem = soup.find("b", string="POVRŠINA")
        if square_m2_elem:
            area_text = square_m2_elem.find_next("div").get_text(strip=True)
            # Extract number and convert to float
            area_match = re.search(r'([\d,\.]+)', area_text)
            if area_match:
                area_str = area_match.group(1).replace(',', '.')
                try:
                    square_m2 = float(area_str)
                except:
                    square_m2 = None
            else:
                square_m2 = None
        else:
            square_m2 = None

        # Extract description
        description_head = soup.find("h3", string=re.compile("Opis nekretnine"))
        description = clean_text(description_head.find_next("p").get_text(" ")) if description_head else None

        # Extract equipment/amenities
        equipment_list = [clean_text(li.get_text()) for li in soup.select("ul.listing-features li")]
        equipment = ", ".join([e for e in equipment_list if e])  # Filter out None values

        details = {
            "title": title,
            "url": url,
            "price_numeric": price_numeric,
            "municipality": municipality,
            "property_type": property_type,
            "ad_type": ad_type,
            "rooms": rooms,
            "square_m2": square_m2,
            "equipment": equipment,
            "description": description
        }

        print("Parsed:", details)
        return details
    except Exception as e:
        print(f"[!] Failed to parse details for {url} → {e}")
        return None


In [ ]:
def create_driver():
    print("[*] Initializing Firefox WebDriver...")
    try:
        options = Options()
        options.binary_location = firefox_binary
        options.add_argument("--headless")

        # ✅ New way to set pageLoadStrategy (Selenium 4+)
        options.set_capability("pageLoadStrategy", "none")

        service = Service(executable_path=geckodriver_binary)
        driver = webdriver.Firefox(service=service, options=options)
        driver.set_page_load_timeout(10)
        print("[+] WebDriver started successfully.")
        return driver
    except Exception as e:
        print(f"[!] Failed to start Firefox driver: {e}")
        return None


## Multithreaded Scraping Functions

We'll use ThreadPoolExecutor to run multiple Selenium instances in parallel. Each thread gets its own WebDriver instance to avoid conflicts.

In [ ]:
def scrape_listing(link, driver):
    """
    Scrape a single listing and return the data.
    Each thread will call this function with its own driver instance.
    """
    try:
        data = parse_detail_page(link, driver)
        if data:
            print(f"      ✔ Scraped: {link}")
        else:
            print(f"      ✖ Failed: {link}")
        time.sleep(random.uniform(*REQUEST_DELAY))
        return data
    except Exception as e:
        print(f"[!] Error scraping {link}: {e}")
        return None


def scrape_page_listings(page_num, driver):
    """
    Scrape all listings from a single search results page.
    Returns a list of listing URLs found on that page.
    """
    print(f"\n[+] Fetching search page {page_num}: {BASE_URL.format(page_num)}")
    html = fetch_page_source(BASE_URL.format(page_num), driver)
    
    if not html:
        print(f"[!] No HTML for page {page_num}, skipping.")
        return []
    
    try:
        soup = BeautifulSoup(html, "lxml")
        links = [urljoin("https://nekretnine.ba/", a["href"]) 
                for a in soup.find_all("a", href=re.compile(r"^real-estate\.php\?lang=ba&sel=nekretnine&view="))]
        
        print(f"  → Found {len(links)} listings on page {page_num}")
        
        if not links:
            print(f"[!] No links found on page {page_num}. Possible structure change?")
        
        return links
    except Exception as e:
        print(f"[!] Failed to parse search page {page_num} → {e}")
        return []


def scrape_with_threading():
    """
    Multithreaded scraping function.
    Creates multiple WebDriver instances and processes listings in parallel.
    """
    fieldnames = ["title", "url", "price_numeric", "municipality", "property_type", "ad_type", "rooms", "square_m2", "equipment", "description"]
    
    # Thread-safe lock for writing to CSV
    csv_lock = threading.Lock()
    
    # Create main driver for collecting listing URLs
    print("[*] Creating main driver for collecting listing URLs...")
    main_driver = create_driver()
    if not main_driver:
        print("[!] Failed to create main driver. Exiting.")
        return
    
    # Collect all listing URLs first
    print(f"[*] Collecting listing URLs from {MAX_PAGES} pages...")
    all_listing_urls = []
    
    for page in range(1, MAX_PAGES + 1):
        links = scrape_page_listings(page, main_driver)
        all_listing_urls.extend(links)
        time.sleep(random.uniform(1, 2))  # Small delay between pages
    
    main_driver.quit()
    print(f"\n[+] Collected {len(all_listing_urls)} total listings to scrape.")
    
    if not all_listing_urls:
        print("[!] No listings found. Exiting.")
        return
    
    # Prepare CSV file
    write_header = not os.path.exists(OUTPUT_CSV)
    
    def worker_scrape(url_batch):
        """Worker function that each thread will execute"""
        driver = create_driver()
        if not driver:
            print("[!] Failed to create worker driver")
            return []
        
        results = []
        for url in url_batch:
            data = scrape_listing(url, driver)
            if data:
                results.append(data)
        
        driver.quit()
        return results
    
    # Split listings into batches for each worker
    batch_size = len(all_listing_urls) // MAX_WORKERS + 1
    url_batches = [all_listing_urls[i:i + batch_size] for i in range(0, len(all_listing_urls), batch_size)]
    
    print(f"\n[*] Starting multithreaded scraping with {MAX_WORKERS} workers...")
    print(f"[*] Processing {len(url_batches)} batches...")
    
    # Use ThreadPoolExecutor for parallel scraping
    all_results = []
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Submit all batches to thread pool
        futures = {executor.submit(worker_scrape, batch): i for i, batch in enumerate(url_batches)}
        
        # Process results as they complete
        for future in as_completed(futures):
            batch_num = futures[future]
            try:
                batch_results = future.result()
                all_results.extend(batch_results)
                print(f"[+] Batch {batch_num + 1}/{len(url_batches)} completed. Scraped {len(batch_results)} listings.")
            except Exception as e:
                print(f"[!] Batch {batch_num + 1} failed: {e}")
    
    # Write all results to CSV at once (thread-safe)
    print(f"\n[*] Writing {len(all_results)} results to CSV...")
    with csv_lock:
        with open(OUTPUT_CSV, "a" if not write_header else "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            if write_header:
                writer.writeheader()
                print(f"[+] Created new CSV file: {OUTPUT_CSV}")
            
            for data in all_results:
                writer.writerow(data)
    
    print(f"\n✅ Finished scraping. Data saved to: {OUTPUT_CSV}")
    print(f"✅ Total listings scraped: {len(all_results)}/{len(all_listing_urls)}")


def scrape():
    """Original single-threaded scraping function (kept for reference)"""
    driver = create_driver()

    fieldnames = ["title", "url", "price_numeric", "municipality", "property_type", "ad_type", "rooms", "square_m2", "equipment", "description"]

    write_header = not os.path.exists(OUTPUT_CSV)
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if write_header:
            writer.writeheader()
            print(f"[+] Created new CSV file: {OUTPUT_CSV}")
        else:
            print(f"[+] Appending to existing CSV: {OUTPUT_CSV}")

        print(f"[*] Starting scraping of up to {MAX_PAGES} pages...")

        for page in range(1, MAX_PAGES + 1):
            print(f"\n[+] Fetching search page {page}: {BASE_URL.format(page)}")
            html = fetch_page_source(BASE_URL.format(page), driver)
            if not html:
                print(f"[!] No HTML for page {page}, skipping.")
                continue

            try:
                soup = BeautifulSoup(html, "lxml")

                links = [urljoin("https://nekretnine.ba/", a["href"]) for a in soup.find_all("a", href=re.compile(r"^real-estate\.php\?lang=ba&sel=nekretnine&view="))]
                
                print(f"  → Found {len(links)} listings on page {page}")

                if not links:
                    print(f"[!] No links found on page {page}. Possible structure change?")
                    continue

                for i, link in enumerate(links, start=1):
                    print(f"    [{i}/{len(links)}] Scraping listing: {link}")
                    try:
                        data = parse_detail_page(link, driver)
                        if data:
                            writer.writerow(data)
                            print("      ✔ Saved listing data to CSV.")
                        else:
                            print("      ✖ No data parsed, skipping.")
                        time.sleep(random.uniform(*REQUEST_DELAY))
                    except Exception as e:
                        print(f"[!] Error scraping {link}: {e}")
            except Exception as e:
                print(f"[!] Failed to parse search page {page} → {e}")

    driver.quit()
    print(f"\n✅ Finished scraping. Data saved to: {OUTPUT_CSV}")

## Run the Scraper

Choose which scraper to run:
- `scrape_with_threading()` - **Multithreaded version** (faster, uses 3 parallel browsers)
- `scrape()` - Single-threaded version (slower, but more stable)

In [ ]:
# if __name__ == "__main__":
#     # Use multithreaded version for faster scraping
#     scrape_with_threading()
    
    # Or use single-threaded version (comment above, uncomment below)
    # scrape()

## 📊 Data Inspection

Let's load and inspect the scraped data from the CSV file.

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../data/sarajevo_flats_nekretnine_cleaned.csv')

# Display basic information
print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
print(f"Total records: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"\nColumn names: {list(df.columns)}")
print("\n" + "=" * 80)

In [ ]:
# Display first few rows
print("FIRST 5 ROWS:")
print("=" * 80)
df.head()

In [ ]:
# Display data types and missing values
print("DATA TYPES AND MISSING VALUES:")
print("=" * 80)
df.info()

In [ ]:
# Statistical summary of numeric columns
print("STATISTICAL SUMMARY (Numeric Columns):")
print("=" * 80)
df.describe()

In [ ]:
# Check for missing values per column
print("MISSING VALUES PER COLUMN:")
print("=" * 80)
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage': missing_percentage.round(2)
})
print(missing_df[missing_df['Missing Count'] > 0])
print("\n" + "=" * 80)

In [ ]:
# Price analysis
print("PRICE ANALYSIS:")
print("=" * 80)
print(f"Average Price: {df['price_numeric'].mean():.2f} KM")
print(f"Median Price: {df['price_numeric'].median():.2f} KM")
print(f"Min Price: {df['price_numeric'].min():.2f} KM")
print(f"Max Price: {df['price_numeric'].max():.2f} KM")
print(f"Standard Deviation: {df['price_numeric'].std():.2f} KM")
print("\n" + "=" * 80)

In [ ]:
# Square meters analysis
print("SQUARE METERS ANALYSIS:")
print("=" * 80)
print(f"Average Area: {df['square_m2'].mean():.2f} m²")
print(f"Median Area: {df['square_m2'].median():.2f} m²")
print(f"Min Area: {df['square_m2'].min():.2f} m²")
print(f"Max Area: {df['square_m2'].max():.2f} m²")
print(f"Standard Deviation: {df['square_m2'].std():.2f} m²")
print("\n" + "=" * 80)

In [ ]:
# Filter out unrealistic property sizes (> 300 m²)
print("FILTERING UNREALISTIC PROPERTY SIZES:")
print("=" * 80)
print(f"Records before filtering: {len(df)}")

# Show properties that will be removed
large_properties = df[df['square_m2'] > 300]
if len(large_properties) > 0:
    print(f"\n⚠️ Found {len(large_properties)} properties with area > 300 m²:")
    print(large_properties[['title', 'square_m2', 'property_type', 'url']])
    
    # Remove properties with area > 300 m²
    df = df[df['square_m2'] <= 300]
    print(f"\n✅ Filtered out {len(large_properties)} properties")
else:
    print("\n✅ No properties with area > 300 m² found")

print(f"Records after filtering: {len(df)}")
print("\n" + "=" * 80)

In [ ]:
# Optionally save the cleaned data to a new CSV file
print("SAVING CLEANED DATA:")
print("=" * 80)
output_file = '../data/sarajevo_flats_nekretnine_cleaned.csv'
df.to_csv(output_file, index=False)
print(f"✅ Cleaned data saved to: {output_file}")
print(f"Total records saved: {len(df)}")
print("\n" + "=" * 80)

In [ ]:
# Price per square meter analysis (filter out invalid data first)
print("PRICE PER SQUARE METER ANALYSIS:")
print("=" * 80)

# Check for zero or null square_m2 values
print(f"Properties with square_m2 = 0 or NaN: {((df['square_m2'] == 0) | df['square_m2'].isna()).sum()}")
print(f"Properties with price_numeric = 0 or NaN: {((df['price_numeric'] == 0) | df['price_numeric'].isna()).sum()}")

# Filter out properties with invalid data for price per m² calculation
valid_df = df[(df['square_m2'] > 0) & (df['price_numeric'] > 0) & df['square_m2'].notna() & df['price_numeric'].notna()].copy()

print(f"\nValid properties for price/m² analysis: {len(valid_df)}/{len(df)}")
print("=" * 80)

# Calculate price per m² only on valid data
valid_df['price_per_m2'] = valid_df['price_numeric'] / valid_df['square_m2']

print(f"\nAverage Price per m²: {valid_df['price_per_m2'].mean():.2f} KM/m²")
print(f"Median Price per m²: {valid_df['price_per_m2'].median():.2f} KM/m²")
print(f"Min Price per m²: {valid_df['price_per_m2'].min():.2f} KM/m²")
print(f"Max Price per m²: {valid_df['price_per_m2'].max():.2f} KM/m²")
print(f"Standard Deviation: {valid_df['price_per_m2'].std():.2f} KM/m²")

# Add price_per_m2 back to main dataframe
df['price_per_m2'] = df.apply(
    lambda row: row['price_numeric'] / row['square_m2'] 
    if (row['square_m2'] > 0 and row['price_numeric'] > 0) 
    else None, 
    axis=1
)

print("\n" + "=" * 80)

In [ ]:
# Check for duplicate records
print("DUPLICATE RECORDS CHECK:")
print("=" * 80)
duplicates = df.duplicated(subset=['url']).sum()
print(f"Number of duplicate URLs: {duplicates}")

if duplicates > 0:
    print("\nDuplicate URLs found:")
    print(df[df.duplicated(subset=['url'], keep=False)][['title', 'url', 'price_numeric']])
else:
    print("No duplicate URLs found!")
    
print("\n" + "=" * 80)

## 📋 Complete Categorical Values Analysis

Let's examine all possible values for each categorical column in the dataset.

In [ ]:
# Identify categorical columns (object dtype or low cardinality numeric columns)
print("=" * 80)
print("ALL UNIQUE VALUES FOR CATEGORICAL COLUMNS")
print("=" * 80)

# Get all columns
all_columns = df.columns.tolist()

# Separate numeric and non-numeric columns
categorical_cols = []
numeric_cols = []

for col in all_columns:
    if df[col].dtype == 'object':
        categorical_cols.append(col)
    elif df[col].dtype in ['int64', 'float64']:
        # Check if it's a low cardinality numeric column (might be categorical)
        unique_count = df[col].nunique()
        if unique_count <= 20:  # Threshold for categorical numeric columns
            categorical_cols.append(col)
        else:
            numeric_cols.append(col)
    else:
        categorical_cols.append(col)

# remove identifier columns from categorical list
categorical_cols = [c for c in categorical_cols if c not in ('title', 'url', 'description', 'equipment')]
print(f"\nFound {len(categorical_cols)} categorical columns")
print(f"Categorical columns: {categorical_cols}")
print(f"\nNumeric columns (excluded): {numeric_cols}")
print(f"\nText columns (excluded): title, url, description, equipment")
print("\n" + "=" * 80)

In [ ]:
# Display all unique values for each categorical column
print("=" * 80)
print("ALL POSSIBLE VALUES FOR EACH CATEGORICAL COLUMN")
print("=" * 80)

for col in categorical_cols:
    print(f"\n{'='*80}")
    print(f"📌 COLUMN: {col.upper()}")
    print(f"{'='*80}")
    
    # Get unique values (excluding NaN)
    unique_values = df[col].dropna().unique()
    unique_count = len(unique_values)
    
    print(f"Total unique values: {unique_count}")
    print(f"\nAll possible values:")
    print("-" * 80)
    
    # Sort values for better readability
    if df[col].dtype in ['int64', 'float64']:
        sorted_values = sorted(unique_values)
    else:
        sorted_values = sorted(unique_values, key=lambda x: str(x))
    
    # Display all unique values in a clean list
    for i, value in enumerate(sorted_values, 1):
        print(f"{i:3d}. {value}")

print("\n" + "=" * 80)
print("✅ ANALYSIS COMPLETE")
print("=" * 80)

## 🗺️ Municipality Standardization

Let's check if addresses or neighborhoods in the data can be mapped to the 9 standard municipalities.

In [ ]:
# Step 1: Analyze current municipality values and look for patterns
print("=" * 80)
print("CURRENT MUNICIPALITY ANALYSIS")
print("=" * 80)

if 'municipality' in df.columns:
    print(f"\nTotal unique municipality values: {df['municipality'].nunique()}")
    print(f"Missing municipality values: {df['municipality'].isnull().sum()}")
    
    print("\n" + "-" * 80)
    print("ALL MUNICIPALITY VALUES (with counts):")
    print("-" * 80)
    
    municipality_counts = df['municipality'].value_counts()
    for idx, (municipality, count) in enumerate(municipality_counts.items(), 1):
        print(f"{idx:3d}. {municipality:<50} ({count} records)")
    
    # Define target municipalities
    target_municipalities = [
        'Hadžići',
        'Ilidža',
        'Ilijaš',
        'Sarajevo - Centar',
        'Sarajevo - Novi Grad',
        'Sarajevo - Novo Sarajevo',
        'Sarajevo - Stari Grad',
        'Trnovo',
        'Vogošća'
    ]
    
    print("\n" + "=" * 80)
    print("TARGET MUNICIPALITIES (from reference):")
    print("=" * 80)
    for idx, municipality in enumerate(target_municipalities, 1):
        count = (df['municipality'] == municipality).sum()
        print(f"{idx:3d}. {municipality:<30} ({count} records in dataset)")
    
    # Check which values don't match target municipalities
    non_matching = df[~df['municipality'].isin(target_municipalities) & df['municipality'].notna()]
    
    if len(non_matching) > 0:
        print("\n" + "=" * 80)
        print(f"⚠️ FOUND {len(non_matching)} RECORDS WITH NON-STANDARD MUNICIPALITY VALUES")
        print("=" * 80)
        print("\nThese need to be mapped to one of the 9 target municipalities:")
        non_matching_counts = non_matching['municipality'].value_counts()
        for municipality, count in non_matching_counts.items():
            print(f"  - {municipality:<50} ({count} records)")
    else:
        print("\n✅ All municipality values already match the target municipalities!")
        
else:
    print("❌ No 'municipality' column found")

print("\n" + "=" * 80)

In [ ]:
# Step 2: Create a comprehensive mapping of neighborhoods to municipalities
print("=" * 80)
print("CREATING NEIGHBORHOOD-TO-MUNICIPALITY MAPPING")
print("=" * 80)

# Define comprehensive mapping based on Sarajevo Canton geography
# This maps neighborhoods, areas, and alternate names to their parent municipalities
neighborhood_to_municipality = {
    # Sarajevo - Centar (Central Sarajevo)
    'Centar': 'Sarajevo - Centar',
    'Marijin Dvor': 'Sarajevo - Centar',
    'Skenderija': 'Sarajevo - Centar',
    'Mejtas': 'Sarajevo - Centar',
    'Mejtaš': 'Sarajevo - Centar',
    'mejtaš': 'Sarajevo - Centar',
    'Džidžikovac': 'Sarajevo - Centar',
    'Bjelave': 'Sarajevo - Centar', 
    'Čobanija': 'Sarajevo - Centar',
    'Šip': 'Sarajevo - Centar',
    'Pearl-Šip': 'Sarajevo - Centar',
    'Parl-Šip': 'Sarajevo - Centar',
    'Koševo': 'Sarajevo - Centar',
    'Koševsko brdo': 'Sarajevo - Centar',
    'Drvenija': 'Sarajevo - Centar',
    'Ferhadija': 'Sarajevo - Centar',
    'Breka': 'Sarajevo - Centar',
    'Soukbunar': 'Sarajevo - Centar',
    
    # Sarajevo - Stari Grad (Old Town)
    'Stari Grad': 'Sarajevo - Stari Grad',
    'Baščaršija': 'Sarajevo - Stari Grad',
    'Alifakovac': 'Sarajevo - Stari Grad',
    'Jekovac': 'Sarajevo - Stari Grad',
    'Kovači': 'Sarajevo - Stari Grad',
    'Vratnik': 'Sarajevo - Stari Grad',
    'Sedrenik': 'Sarajevo - Stari Grad',
    'Hrid': 'Sarajevo - Stari Grad',
    'Bistrik': 'Sarajevo - Stari Grad',
    
    # Sarajevo - Novo Sarajevo (New Sarajevo)
    'Novo Sarajevo': 'Sarajevo - Novo Sarajevo',
    'Grbavica': 'Sarajevo - Novo Sarajevo',
    'Dolac Malta': 'Sarajevo - Novo Sarajevo',
    'Ciglane': 'Sarajevo - Novo Sarajevo',
    'Hrasno': 'Sarajevo - Novo Sarajevo',
    'Velešići': 'Sarajevo - Novo Sarajevo',
    'Kovačići': 'Sarajevo - Novo Sarajevo',
    'Kovacici': 'Sarajevo - Novo Sarajevo',
    'Vraca': 'Sarajevo - Novo Sarajevo',
    'Zmaja od Bosne': 'Sarajevo - Novo Sarajevo',
    'Pofalići': 'Sarajevo - Novo Sarajevo',
    'Socijalno': 'Sarajevo - Novo Sarajevo',
    'Robot Socijalno': 'Sarajevo - Novo Sarajevo',
    'Sarajevo Tower': 'Sarajevo - Novo Sarajevo',
    
    # Sarajevo - Novi Grad (New City)
    'Novi Grad': 'Sarajevo - Novi Grad',
    'Švrakino Selo': 'Sarajevo - Novi Grad',
    'Alipašino Polje': 'Sarajevo - Novi Grad',
    'Alipašino': 'Sarajevo - Novi Grad',
    'Čengić Vila': 'Sarajevo - Novi Grad',
    'Zabrđe': 'Sarajevo - Novi Grad',
    'Stupsko Brdo': 'Sarajevo - Novi Grad',
    'Buća Potok': 'Sarajevo - Novi Grad',
    'Vojničko polje': 'Sarajevo - Novi Grad',
    'Mali Bosmal': 'Sarajevo - Novi Grad',
    'Fra Antuna Kneževića': 'Sarajevo - Novi Grad',
    'Aerodromsko naslje': 'Sarajevo - Novi Grad',
    'Miljacka': 'Sarajevo - Novi Grad',
    'Bulevar': 'Sarajevo - Novi Grad',
    'Teheranski trg': 'Sarajevo - Novi Grad',
    'Mojmilo': 'Sarajevo - Novi Grad',
    'Dobrinja': 'Sarajevo - Novi Grad',
    'Otoka': 'Sarajevo - Novi Grad',
    'Aneks': 'Sarajevo - Novi Grad',
    
    
    # Ilidža
    'Ilidža': 'Ilidža',
    'Butmir': 'Ilidža',
    'Sokolović Kolonija': 'Ilidža',
    'Otes': 'Ilidža',
    'Pejton': 'Ilidža',
    'Hrasnica': 'Ilidža',
    'Blažuj': 'Ilidža',
    'Lužani': 'Ilidža',
    'Luzani': 'Ilidža',
    'luzani': 'Ilidža',
    'Pijacna': 'Ilidža',
    'Stup': 'Ilidža',
    'Gray Residence': 'Ilidža',
    
    
    # Hadžići
    'Hadžići': 'Hadžići',
    'Pazarić': 'Hadžići',
    'Tarčin': 'Hadžići',
    'Toviš': 'Hadžići',
    
    # Vogošća
    'Vogošća': 'Vogošća',
    'Semizovac': 'Vogošća',
    'Kobilja Glava': 'Vogošća',
    'Vogoscanskih odreda': 'Vogošća',
    'Vogošćanskih odreda': 'Vogošća',
    'Hotonj': 'Vogošća',
    
    # Ilijaš
    'Ilijaš': 'Ilijaš',
    'Podlugovi': 'Ilijaš',
    
    # Trnovo
    'Trnovo': 'Trnovo',
    'Trnovo - Bjelašnica': 'Trnovo',
    'Bjelašnica': 'Trnovo',
    'Artes Bjelašnica': 'Trnovo',
    
    # NOTE: Some locations are outside Sarajevo Canton:
    # - Lukavica (East Sarajevo - not in Sarajevo Canton)
    # - Jahorina (Pale municipality - not in Sarajevo Canton)
    # - Ravna planina (part of Jahorina area)
    # - Makarska (coastal city in Croatia)
    # - Visoko (separate municipality, not in Sarajevo Canton)
    # These will remain unmapped as they don't belong to the 9 target municipalities
    
    # Street names that can be mapped based on known locations:
    'Džemala Bijedića': 'Sarajevo - Novi Grad',
    'Dr. Silve Rizvanbegović': 'Sarajevo - Centar',
    'Dr.Silve Rizvanbegovic': 'Ilidža',
    'Silve Rizvanbegovic': 'Ilidža',
    'Josipa Slavenskog': 'Ilidža',
    'Mesa Selimovic': 'Sarajevo - Novi Grad',
    'Svetozara Ćorovića': 'Sarajevo - Centar',
    'Semira Fraste': 'Sarajevo - Novi Grad',
    'F. Becirbegovica': 'Sarajevo - Novo Sarajevo',
    'Tome Mendesa': 'Vogošća',
    'Ante Babića': 'Sarajevo - Novi Grad',
    'Ibrahima Ljubovica': 'Ilidža',
    'Samira Catovica Kobre': 'Ilidža',
    'Ramiza Jasara': 'Ilidža',
    'ramiza jasara': 'Ilidža',
    'Karla Malya': 'Ilidža',
    'Trg solidarnosti': 'Sarajevo - Novi Grad',
    'Nikole Sopa': 'Ilidža',
    'Skendera Kulenovica': 'Sarajevo - Stari Grad',
    'Hifzi Bjelavca': 'Ilidža',
    'Bešarevića': 'Sarajevo - Centar',
    'Samin gaj': 'Ilidža',
    'Slatina': 'Sarajevo - Centar',
    'Kod OHR-a': 'Sarajevo - Centar',
    'Latička': 'Ilidža',
    'Stupska': 'Ilidža',
    'p.o.zvijezda': 'Vogošća',
}

print(f"\n✅ Created mapping with {len(neighborhood_to_municipality)} neighborhood entries")
print(f"   Mapping to {len(set(neighborhood_to_municipality.values()))} municipalities")

print("\n" + "-" * 80)
print("SAMPLE MAPPINGS:")
print("-" * 80)
for i, (neighborhood, municipality) in enumerate(list(neighborhood_to_municipality.items())[:10], 1):
    print(f"{i:3d}. {neighborhood:<30} → {municipality}")

print("\n" + "=" * 80)

In [ ]:
# Step 3: Check if title or municipality contains neighborhood keywords
print("=" * 80)
print("SEARCHING FOR NEIGHBORHOODS IN MUNICIPALITY AND TITLE COLUMNS")
print("=" * 80)

if 'municipality' in df.columns:
    # Create a copy of the original municipality column for comparison
    df['municipality_original'] = df['municipality'].copy()
    
    found_mappings = []
    unmapped_records = []
    
    for idx, row in df.iterrows():
        municipality_value = str(row['municipality']) if pd.notna(row['municipality']) else ''
        title_value = str(row['title']) if pd.notna(row['title']) else ''
        
        # Combine both for searching
        search_text = f"{municipality_value} {title_value}".lower()
        
        # Try to find a matching neighborhood
        matched = False
        for neighborhood, target_municipality in neighborhood_to_municipality.items():
            if neighborhood.lower() in search_text:
                # Found a match!
                if pd.isna(row['municipality']) or row['municipality'] != target_municipality:
                    found_mappings.append({
                        'index': idx,
                        'original_municipality': row['municipality'],
                        'title': row['title'],
                        'found_neighborhood': neighborhood,
                        'mapped_to': target_municipality
                    })
                    # Update the municipality
                    df.at[idx, 'municipality'] = target_municipality
                matched = True
                break
        
        # If no match found and municipality is not one of the target 9
        target_municipalities = set(neighborhood_to_municipality.values())
        if not matched and pd.notna(row['municipality']) and row['municipality'] not in target_municipalities:
            unmapped_records.append({
                'index': idx,
                'municipality': row['municipality'],
                'title': row['title']
            })
    
    print(f"\n✅ FOUND {len(found_mappings)} records with neighborhood keywords")
    
    if len(found_mappings) > 0:
        print("\n" + "-" * 80)
        print("SAMPLE OF MAPPED RECORDS (first 10):")
        print("-" * 80)
        for i, mapping in enumerate(found_mappings[:10], 1):
            print(f"\n{i}. Found '{mapping['found_neighborhood']}' → Mapped to '{mapping['mapped_to']}'")
            print(f"   Original: {mapping['original_municipality']}")
            print(f"   Title: {mapping['title'][:80]}...")
    
    if len(unmapped_records) > 0:
        print("\n" + "=" * 80)
        print(f"⚠️ STILL HAVE {len(unmapped_records)} UNMAPPED RECORDS")
        print("=" * 80)
        print("\nThese don't match any known neighborhood:")
        
        # Show unique unmapped municipalities
        unique_unmapped = {}
        for record in unmapped_records:
            mun = record['municipality']
            if mun not in unique_unmapped:
                unique_unmapped[mun] = []
            unique_unmapped[mun].append(record['title'])
        
        for mun, titles in unique_unmapped.items():
            print(f"\n  {mun} ({len(titles)} records)")
            print(f"    Sample title: {titles[0][:80]}...")
    else:
        print("\n✅ All records successfully mapped!")
        
else:
    print("❌ No 'municipality' column found")

print("\n" + "=" * 80)

In [ ]:
# Step 4: Show final municipality distribution after mapping
print("=" * 80)
print("FINAL MUNICIPALITY DISTRIBUTION AFTER MAPPING")
print("=" * 80)

if 'municipality' in df.columns:
    print("\nMunicipality Value Counts:")
    print("-" * 80)
    
    final_counts = df['municipality'].value_counts()
    for idx, (municipality, count) in enumerate(final_counts.items(), 1):
        percentage = (count / len(df)) * 100
        print(f"{idx:3d}. {municipality:<35} {count:5d} records ({percentage:5.2f}%)")
    
    # Check how many were successfully mapped
    target_municipalities = [
        'Hadžići', 'Ilidža', 'Ilijaš', 'Sarajevo - Centar',
        'Sarajevo - Novi Grad', 'Sarajevo - Novo Sarajevo',
        'Sarajevo - Stari Grad', 'Trnovo', 'Vogošća'
    ]
    
    mapped_count = df[df['municipality'].isin(target_municipalities)].shape[0]
    unmapped_count = df[~df['municipality'].isin(target_municipalities) & df['municipality'].notna()].shape[0]
    missing_count = df['municipality'].isnull().sum()
    
    print("\n" + "=" * 80)
    print("MAPPING SUMMARY:")
    print("=" * 80)
    print(f"✅ Mapped to target municipalities: {mapped_count} ({mapped_count/len(df)*100:.2f}%)")
    print(f"⚠️  Still unmapped (non-standard):   {unmapped_count} ({unmapped_count/len(df)*100:.2f}%)")
    print(f"❌ Missing municipality data:       {missing_count} ({missing_count/len(df)*100:.2f}%)")
    print(f"\n📊 Total records: {len(df)}")
    
    # Show what changed
    if 'municipality_original' in df.columns:
        changed = df[df['municipality'] != df['municipality_original']].shape[0]
        print(f"\n🔄 Updated {changed} records with neighborhood mapping")
    
else:
    print("❌ No 'municipality' column found")

print("\n" + "=" * 80)

In [ ]:
# Step 5: Remove records outside Sarajevo Canton
print("=" * 80)
print("REMOVING RECORDS OUTSIDE SARAJEVO CANTON")
print("=" * 80)

# Define keywords for locations outside Sarajevo Canton
outside_canton_keywords = [
    'lukavica', 'lukavici', 'jahorina', 'jahoirina', 'ravna planina', 
    'makarska', 'visoko', 'dvorišta', 'vučko', 'pahulja', 
    'olovske luke', 'ajdinovići', 'azapovići', 'ponijeri', 'podvisoki', 
    'homolj', 'prhinje', 'boška jugovića', 'dr. džananovića', 
    'spasovdanska', 'srpskih vladara', 'bukova ravan', 'tušnjići', 
    'bešagića visoko'
]

print(f"\nSearching for {len(outside_canton_keywords)} location keywords...")
print(f"Keywords: {', '.join(outside_canton_keywords[:5])}... (and {len(outside_canton_keywords) - 5} more)")

# Track which records to remove
records_to_remove = []
keyword_counts = {keyword: [] for keyword in outside_canton_keywords}

initial_count = len(df)

# Search in both municipality and title columns (case-insensitive)
for idx, row in df.iterrows():
    municipality_text = str(row['municipality']).lower() if pd.notna(row['municipality']) else ''
    title_text = str(row['title']).lower() if pd.notna(row['title']) else ''
    search_text = f"{municipality_text} {title_text}"
    
    for keyword in outside_canton_keywords:
        if keyword in search_text:
            records_to_remove.append(idx)
            keyword_counts[keyword].append({
                'index': idx,
                'municipality': row['municipality'],
                'title': row['title'][:80]
            })
            break  # Only count each record once

print("\n" + "-" * 80)
print("FOUND RECORDS TO REMOVE:")
print("-" * 80)

# Show breakdown by keyword
records_found = 0
for keyword, records in keyword_counts.items():
    if len(records) > 0:
        records_found += len(records)
        print(f"\n🔍 Keyword '{keyword}': {len(records)} records")
        for i, record in enumerate(records[:3], 1):  # Show first 3 examples
            print(f"   {i}. {record['municipality']} - {record['title']}")
        if len(records) > 3:
            print(f"   ... and {len(records) - 3} more")

if records_found > 0:
    # Remove the records
    indices_to_drop = list(set(records_to_remove))  # Remove duplicates
    df.drop(indices_to_drop, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    final_count = len(df)
    removed_count = initial_count - final_count
    
    print("\n" + "=" * 80)
    print("REMOVAL SUMMARY:")
    print("=" * 80)
    print(f"✅ Removed {removed_count} records outside Sarajevo Canton")
    print(f"📊 Records before: {initial_count}")
    print(f"📊 Records after:  {final_count}")
    print(f"📈 Removed: {(removed_count/initial_count)*100:.2f}%")
else:
    print("\n✅ No records found outside Sarajevo Canton")

print("\n" + "=" * 80)

## 📝 Extract Municipalities from Descriptions

For records with missing municipality data, let's search their descriptions for mentions of neighborhoods or municipalities.

In [ ]:
# Step 6: Extract municipalities from descriptions for records with missing municipality data
print("=" * 80)
print("EXTRACTING MUNICIPALITIES FROM DESCRIPTIONS")
print("=" * 80)

if 'municipality' in df.columns and 'description' in df.columns:
    # Find records with missing municipality but have description
    missing_municipality = df['municipality'].isnull()
    has_description = df['description'].notna()
    
    records_to_check = missing_municipality & has_description
    records_count = records_to_check.sum()
    
    print(f"\n📊 Records with missing municipality: {missing_municipality.sum()}")
    print(f"📊 Records with description available: {has_description.sum()}")
    print(f"🔍 Records to check (missing municipality + has description): {records_count}")
    
    if records_count > 0:
        print("\n" + "-" * 80)
        print("SEARCHING DESCRIPTIONS FOR MUNICIPALITY KEYWORDS...")
        print("-" * 80)
        
        # Use the same neighborhood mapping dictionary
        found_from_description = []
        
        for idx, row in df[records_to_check].iterrows():
            description_text = str(row['description']).lower() if pd.notna(row['description']) else ''
            title_text = str(row['title']).lower() if pd.notna(row['title']) else ''
            
            # Combine description and title for better matching
            search_text = f"{description_text} {title_text}"
            
            # Try to find a matching neighborhood in the description
            matched = False
            for neighborhood, target_municipality in neighborhood_to_municipality.items():
                if neighborhood.lower() in search_text:
                    # Found a match!
                    found_from_description.append({
                        'index': idx,
                        'title': row['title'],
                        'found_neighborhood': neighborhood,
                        'mapped_to': target_municipality,
                        'description_snippet': row['description'][:100] if pd.notna(row['description']) else ''
                    })
                    # Update the municipality
                    df.at[idx, 'municipality'] = target_municipality
                    matched = True
                    break
        
        if len(found_from_description) > 0:
            print(f"\n✅ SUCCESS! Found {len(found_from_description)} municipalities from descriptions!")
            print("\n" + "-" * 80)
            print("SAMPLE OF EXTRACTED MUNICIPALITIES (first 10):")
            print("-" * 80)
            
            for i, extraction in enumerate(found_from_description[:10], 1):
                print(f"\n{i}. Found '{extraction['found_neighborhood']}' → Mapped to '{extraction['mapped_to']}'")
                print(f"   Title: {extraction['title'][:70]}...")
                print(f"   Description: {extraction['description_snippet']}...")
            
            if len(found_from_description) > 10:
                print(f"\n   ... and {len(found_from_description) - 10} more")
            
            # Show updated statistics
            print("\n" + "=" * 80)
            print("UPDATED MUNICIPALITY STATISTICS:")
            print("=" * 80)
            
            target_municipalities = [
                'Hadžići', 'Ilidža', 'Ilijaš', 'Sarajevo - Centar',
                'Sarajevo - Novi Grad', 'Sarajevo - Novo Sarajevo',
                'Sarajevo - Stari Grad', 'Trnovo', 'Vogošća'
            ]
            
            mapped_count = df[df['municipality'].isin(target_municipalities)].shape[0]
            missing_count = df['municipality'].isnull().sum()
            
            print(f"\n✅ Mapped to target municipalities: {mapped_count} ({mapped_count/len(df)*100:.2f}%)")
            print(f"❌ Still missing municipality data: {missing_count} ({missing_count/len(df)*100:.2f}%)")
            print(f"📈 Improvement: +{len(found_from_description)} records mapped from descriptions")
            print(f"\n📊 Total records: {len(df)}")
            
            # Show final distribution by municipality
            print("\n" + "-" * 80)
            print("MUNICIPALITY DISTRIBUTION (after description extraction):")
            print("-" * 80)
            
            final_counts = df['municipality'].value_counts()
            for idx, (municipality, count) in enumerate(final_counts.items(), 1):
                if municipality in target_municipalities:
                    percentage = (count / len(df)) * 100
                    print(f"{idx:3d}. {municipality:<35} {count:5d} records ({percentage:5.2f}%)")
        else:
            print("\n⚠️  No additional municipalities found in descriptions")
            print("    The descriptions may not contain explicit neighborhood/municipality names")
    else:
        print("\n✅ No records need description-based extraction")
        print("    All records either have municipality data or lack descriptions")
else:
    print("❌ Required columns not found")

print("\n" + "=" * 80)

In [ ]:
# # Update the cleaned CSV file without property_type and ad_type columns
# print("UPDATING CLEANED DATA FILE:")
# print("=" * 80)
# output_file = '../data/sarajevo_flats_nekretnine_cleaned_1.csv'
# df.to_csv(output_file, index=False)
# print(f"✅ Updated cleaned data saved to: {output_file}")
# print(f"Total columns: {len(df.columns)}")
# print(f"Column names: {list(df.columns)}")
# print("\n" + "=" * 80)